In [ ]:
import numpy as np
import scipp as sc
import sciline
import scippneutron as scn
import plopp
import esssans as sans
import esssans.isis
from esssans.types import *

In [ ]:
root = '/home/simon/instruments/zoom/ZOOM_Scipp'
params = {
    sans.isis.IDFFilename: '/home/simon/mambaforge/envs/mantid310/instrument/ZOOM_Definition.xml',
    sans.types.DirectBeamFilename: f'{root}/Direct_Zoom_4m_8mm_100522.txt',
    sans.isis.Filename[sans.types.SampleRun]: f'{root}/ZOOM00034786.nxs',
    sans.isis.Filename[sans.types.EmptyBeamRun]: f'{root}/ZOOM00034787.nxs',
}
masks = ['small_bs_232.xml', 'tube_1120_bottom.xml']
masks = ['small_bs_232.xml']
mask_paths = [f'{root}/{mask}' for mask in masks]
masks = sciline.ParamTable(str, {sans.isis.PixelMaskFilename: mask_paths}, index=masks)

In [ ]:
def get_transmission_data(
    data: LoadedFileContents[SampleRun],
) -> LoadedFileContents[TransmissionRun[SampleRun]]:
    return data

In [ ]:
# TODO Check monitor names
params[NeXusMonitorName[Incident]] = 'monitor3'
params[NeXusMonitorName[Transmission]] = 'monitor5'
# No separate transmission run
# params[LoadedFileContents[TransmissionRun[SampleRun]]] = params[
#    LoadedFileContents[SampleRun]
# ]

# TODO How to load calibration
# TODO How to load or apply masks?
# TODO What is the moderator file?


band = sc.linspace('wavelength', 1.75, 16.5, num=2, unit='angstrom')
params[WavelengthBands] = band
params[WavelengthBins] = sc.linspace(
    'wavelength', start=band[0], stop=band[-1], num=141
)

# mask_interval = sc.array(dims=['wavelength'], values=[2.21, 2.59], unit='angstrom')
# params[WavelengthMask] = sc.DataArray(
#     sc.array(dims=['wavelength'], values=[True]),
#     coords={'wavelength': mask_interval},
# )

params[QBins] = sc.linspace(dim='Q', start=0.004, stop=0.8, num=141, unit='1/angstrom')
params[NonBackgroundWavelengthRange] = sc.array(
    dims=['wavelength'], values=[0.7, 17.1], unit='angstrom'
)
params[CorrectForGravity] = True
params[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound

In [ ]:
providers = (
    sans.providers
    + sans.isis.providers
    + (get_transmission_data,)
    + sans.sans2d.general_providers
)
pipeline = sciline.Pipeline(providers, params=params)
pipeline.set_param_table(masks)

In [ ]:
pipeline.visualize(MaskedData[SampleRun], graph_attr={'rankdir': 'LR'})

In [ ]:
pipeline.compute(MaskedData[SampleRun])

In [ ]:
iofq = pipeline.get(IofQ[SampleRun])
iofq.visualize(graph_attr={'rankdir': 'LR'})

In [ ]:
%matplotlib widget
da = iofq.compute()
da.plot()

To do:
- beam center finder flatten issue
- zoom files via scippneutron.load_with_mantid
- load masks with mantid?
- `direct_beam = loki.io.load_rkh_wav(f'{path}/DirectBeam_20feb_full_v3.dat')`
- make providers from mantid loaders
- add files to pooch for testing?
```
mask_file = f'{path}/mask_new_July2022.xml'
mask_all = scn.load(filename=idf_filename, mantid_alg="LoadMask", mantid_args={"InputFile": mask_file})
```

In [ ]:
root = '/home/simon/instruments/zoom/ZOOM_Scipp'

run86 = sc.io.load_hdf5(f'{root}/ZOOM00034786.nxs.hdf5')
run87 = sc.io.load_hdf5(f'{root}/ZOOM00034787.nxs.hdf5')
cal = sc.io.load_hdf5(f'{root}/192tubeCalibration_11-02-2019_r5_10lines.nxs.hdf5')

run86['data'].coords["gravity"] = sans.common.gravity_vector()

In [ ]:
# da = pipeline.get(RawData[SampleRun]).compute()
da = pipeline.get(CalibratedMaskedData[SampleRun]).compute()
scn.instrument_view(da.hist(), pixel_size=0.01, norm='log')

In [ ]:
da = pipeline.get(RawData[BackgroundRun]).compute()
scn.instrument_view(da.hist(), pixel_size=0.01, norm='log')

In [ ]:
# iofq = pipeline.get(BackgroundSubtractedIofQ)
iofq = pipeline.get(IofQ[SampleRun])

### Compute intermediate results

For inspection and debugging purposes we can also compute intermediate results.
To avoid repeated computation (including costly loading of files) we can request multiple results at once, including the final result, if desired.
For example:

In [ ]:
monitors = (
    WavelengthMonitor[SampleRun, Incident],
    WavelengthMonitor[SampleRun, Transmission],
    # WavelengthMonitor[BackgroundRun, Incident],
    # WavelengthMonitor[BackgroundRun, Transmission],
)
parts = (CleanSummedQ[SampleRun, Numerator], CleanSummedQ[SampleRun, Denominator])
iofqs = (IofQ[SampleRun],)
keys = monitors + (MaskedData[SampleRun],) + parts + iofqs

results = pipeline.compute(keys)

display(sc.plot({str(key): results[key] for key in monitors}, norm='log'))

display(
    scn.instrument_view(
        results[MaskedData[SampleRun]].hist(),
        pixel_size=0.0075,
        norm='log',
        camera=pp.graphics.Camera(position=(0, 0, 22)),
    )
)

parts = {str(key): results[key] for key in parts}
parts = {key: val if val.bins is None else val.hist() for key, val in parts.items()}
display(sc.plot(parts, norm='log'))

iofqs = {str(key): results[key] for key in iofqs}
iofqs = {key: val if val.bins is None else val.hist() for key, val in iofqs.items()}
display(sc.plot(iofqs, norm='log'))